# Swiss ADME Feature creation for each Cytochrome P450 enzyme
Data dictionary in the README has the feature data



### Imports

In [1]:
import pandas as pd
import numpy as np
import os

from rdkit import Chem, DataStructs
from rdkit.Chem import rdchem
from rdkit.Chem import Mol
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import FilterCatalog
from rdkit.Chem import rdqueries

from sklearn.model_selection import train_test_split

from collections import defaultdict

import warnings
warnings.filterwarnings('ignore')

<frozen importlib._bootstrap>:219: RuntimeWarning: to-Python converter for boost::shared_ptr<RDKit::FilterCatalogEntry const> already registered; second conversion method ignored.


### Import Data

In [2]:
# Import data and remove unnecessary header rows
cyto_assay = pd.read_csv('.././data/train_data/cyto_assay_clean.csv', skipinitialspace=True, header=[0,4])
smiles_merged = pd.read_pickle('.././data/conversion_data/smiles_merged.pkl')

Panel Name KEY  

* 'p450-cyp2c19' : 0
* 'p450-cyp2c9'  : 1
* 'p450-cyp2d6'  : 2
* 'p450-cyp1a2'  : 3
* 'p450-cyp3a4'  : 4 

# Feature creation functions

In [3]:
# adapted from src/cyp_chembl_et.ipynb in this project
class Features(object):
    
    def __init__(self, mol):
        
        if isinstance(mol, str):
            self.mol = Chem.MolFromSmiles(mol)
        else:
            self.mol = mol
            
    def h_bond_donors(self):
        return Chem.Lipinski.NumHDonors(self.mol)

    def h_bond_acceptors(self):
        return Chem.Lipinski.NumHAcceptors(self.mol)

    def molar_refractivity(self):
        return Chem.Crippen.MolMR(self.mol)

    def molecular_weight(self):
        return Descriptors.ExactMolWt(self.mol)

    def n_atoms(self):
        return self.mol.GetNumAtoms()

    def n_carbons(self):
        carbon = Chem.rdqueries.AtomNumEqualsQueryAtom(6)
        return len(self.mol.GetAtomsMatchingQuery(carbon))

    def n_heteroatoms(self):
        return Descriptors.rdMolDescriptors.CalcNumHeteroatoms(self.mol)

    def n_rings(self):
        return Descriptors.rdMolDescriptors.CalcNumRings(self.mol)

    def n_rot_bonds(self):
        return Chem.Lipinski.NumRotatableBonds(self.mol)

    def logp(self):
        return Descriptors.MolLogP(self.mol)

    def tpsa(self):
        return Descriptors.TPSA(self.mol)
    
    def bond_type(bond):
        return Chem.rdchem.Bond.GetBondType(bond)
    
    def is_conjugated(bond):
        return Chem.rdchem.Bond.GetIsConjugated(bond)
        
    def in_ring(bond):
        return Chem.rdchem.Atom.IsInRing(bond)
        
    def stereo(bond):
        return Chem.rdchem.Bond.GetStereo(bond)
    
    def get_atoms(self):
        return Chem.rdchem.Mol.GetAtoms(self.mol)
    
    def get_bonds(self, atom):
        return Chem.rdchem.Atom.GetBonds(atom)
    
    def conformer(self):
        return Chem.rdchem.Conformer()
    
    def n_bonds(self):
        d = defaultdict(int)
        atoms = self.get_atoms()
        for atom in atoms:
            bonds = self.get_bonds(atom)
            for bond in bonds:
                d[Chem.rdchem.Bond.GetBondType(bond)] += 1
        return d
    
    def n_heavy_atoms(self):
        return Chem.rdchem.Mol.GetNumHeavyAtoms(self.mol)
    
    def n_aromatic_atom(self):
        count = 0
        aromatic = Chem.rdchem.Mol.GetAromaticAtoms(self.mol)
        for atom in aromatic:
            count += 1 
        return count
    
#     def w_sum_carb_hal(mol):
#         q = rdqueries.AtomNumEqualsQueryAtom(6)
#         return len(mol.GetAtomsMatchingQuery(q))
    
    

In [4]:
# Function to create features

def swiss_feat(mol_list):

    array = ['h_bond_donors', 'h_bond_acceptors', 'molar_refractivity', 'molecular_weight', 'n_atoms','n_carbons',
                 'n_heteroatoms', 'n_rings', 'n_rot_bonds', 'logp', 'tpsa', 'n_heavy_atoms', 'n_aromatic_atom']
        
    extra_col = ['single_bond', 'double_bond', 'triple_bond', 'aromatic_bond']

    feature_df = pd.DataFrame([], columns=array+extra_col)

    for i in mol_list:
        mol = Features(i)

        feature_arr = []
        for i in array:
            val = getattr(mol, i)()
            feature_arr.append(val)
        num_bonds = mol.n_bonds()
        feature_arr.append(num_bonds[Chem.rdchem.BondType.SINGLE])
        feature_arr.append(num_bonds[Chem.rdchem.BondType.DOUBLE])
        feature_arr.append(num_bonds[Chem.rdchem.BondType.TRIPLE])
        feature_arr.append(num_bonds[Chem.rdchem.BondType.AROMATIC])
        feature_df = feature_df.append(dict(zip(feature_df.columns, feature_arr)), ignore_index=True)
    return feature_df

## CYP2c19

In [5]:
# CYP2c19 is mapped to the 0th value
cyp2c19_smiles_merge = smiles_merged[smiles_merged['Panel Name'] == 0]

#Isolate the SMILES data to be converted
cyp2c19_smiles = cyp2c19_smiles_merge[['SMILES']]

# Convert MolFromSmiles to GetMorganFingerprintAsBitVect to ToBitString to get bitstrings for SMILES data
cyp2c19_mol = cyp2c19_smiles.apply(lambda row: Chem.MolFromSmiles(row['SMILES']), axis=1)
cyp2c19_mol = cyp2c19_mol.to_list()

# Set up mol dataset with index
cyp2c19_mol = swiss_feat(cyp2c19_mol)
cyp2c19_mol = cyp2c19_mol.reset_index()

print(type(cyp2c19_mol))

#combine datasets for modeling
cyp2c19_smiles_merge = cyp2c19_smiles_merge[['index', 'Inhibition Observed', 'Panel Name']]
cyp2c19_swiss_feat = cyp2c19_smiles_merge.merge(cyp2c19_mol, how="inner", on="index")

# Save to data/cyp_datasets
cyp2c19_swiss_feat.to_pickle('.././data/cyp_datasets/cyp2c19_swiss_feat.pkl')
cyp2c19_swiss_feat.head()

<class 'pandas.core.frame.DataFrame'>


,index,Inhibition Observed,Panel Name,h_bond_donors,h_bond_acceptors,molar_refractivity,molecular_weight,n_atoms,n_carbons,n_heteroatoms,n_rings,n_rot_bonds,logp,tpsa,n_heavy_atoms,n_aromatic_atom,single_bond,double_bond,triple_bond,aromatic_bond
0,5,1,0,1.0,4.0,70.5419,253.113650,17.0,13.0,4.0,2.0,3.0,3.16440,52.32,17.0,5.0,24.0,2.0,0.0,10.0
1,10,1,0,0.0,4.0,87.3980,315.059696,21.0,16.0,5.0,3.0,4.0,4.23340,30.71,21.0,17.0,12.0,0.0,0.0,34.0
2,15,1,0,1.0,2.0,61.9434,297.960311,14.0,10.0,4.0,1.0,2.0,1.68348,66.88,14.0,6.0,10.0,4.0,2.0,12.0
3,20,1,0,2.0,5.0,54.6820,222.075290,16.0,9.0,7.0,2.0,2.0,-0.01338,100.35,16.0,9.0,10.0,4.0,0.0,20.0
4,25,1,0,1.0,2.0,72.1642,295.082013,21.0,15.0,6.0,2.0,4.0,3.98782,38.33,21.0,12.0,18.0,2.0,0.0,24.0


## CYP2c9

In [6]:
# CYP2c19 is mapped to the 1st value
cyp2c9_smiles_merge = smiles_merged[smiles_merged['Panel Name'] == 1]

#Isolate the SMILES data to be converted
cyp2c9_smiles = cyp2c9_smiles_merge[['SMILES']]

# Convert MolFromSmiles to GetMorganFingerprintAsBitVect to ToBitString to get bitstrings for SMILES data
cyp2c9_mol = cyp2c9_smiles.apply(lambda row: Chem.MolFromSmiles(row['SMILES']), axis=1)
cyp2c9_mol = cyp2c9_mol.to_list()

# Set up mol dataset with index
cyp2c9_mol = swiss_feat(cyp2c9_mol)
cyp2c9_mol = cyp2c9_mol.reset_index()

print(type(cyp2c9_mol))

#combine datasets for modeling
cyp2c9_smiles_merge = cyp2c9_smiles_merge[['index', 'Inhibition Observed', 'Panel Name']]
cyp2c9_swiss_feat = cyp2c9_smiles_merge.merge(cyp2c9_mol, how="inner", on="index")

# Save to data/cyp_datasets
cyp2c9_swiss_feat.to_pickle('.././data/cyp_datasets/cyp2c9_swiss_feat.pkl')
cyp2c9_swiss_feat.head()

<class 'pandas.core.frame.DataFrame'>


,index,Inhibition Observed,Panel Name,h_bond_donors,h_bond_acceptors,molar_refractivity,molecular_weight,n_atoms,n_carbons,n_heteroatoms,n_rings,n_rot_bonds,logp,tpsa,n_heavy_atoms,n_aromatic_atom,single_bond,double_bond,triple_bond,aromatic_bond
0,9,1,1,1.0,4.0,78.6186,304.085935,22.0,15.0,7.0,2.0,5.0,3.38490,81.47,22.0,12.0,18.0,4.0,0.0,24.0
1,14,1,1,1.0,4.0,84.9812,332.084017,23.0,16.0,7.0,3.0,5.0,2.57970,59.29,23.0,15.0,16.0,2.0,0.0,32.0
2,19,1,1,2.0,3.0,82.2436,328.085719,22.0,15.0,7.0,2.0,3.0,4.47222,55.12,22.0,11.0,22.0,2.0,0.0,22.0
3,24,1,1,1.0,7.0,78.5761,326.089416,22.0,12.0,10.0,2.0,6.0,1.32702,107.88,22.0,10.0,22.0,4.0,0.0,20.0
4,29,1,1,2.0,7.0,78.8974,306.089895,21.0,12.0,9.0,2.0,5.0,0.89930,101.80,21.0,11.0,18.0,4.0,0.0,22.0


## CYP2d6

In [7]:
# CYP2c19 is mapped to the 2nd value
cyp2d6_smiles_merge = smiles_merged[smiles_merged['Panel Name'] == 2]
cyp2d6_smiles = cyp2d6_smiles_merge[['SMILES']]

# Convert MolFromSmiles to GetMorganFingerprintAsBitVect to ToBitString to get bitstrings for SMILES data
cyp2d6_mol = cyp2d6_smiles.apply(lambda row: Chem.MolFromSmiles(row['SMILES']), axis=1)
cyp2d6_mol = cyp2d6_mol.to_list()

# Set up mol dataset with index
cyp2d6_mol = swiss_feat(cyp2d6_mol)
cyp2d6_mol = cyp2d6_mol.reset_index()

print(type(cyp2c9_mol))

#combine datasets for modeling
cyp2d6_smiles_merge = cyp2d6_smiles_merge[['index', 'Inhibition Observed', 'Panel Name']]
cyp2d6_swiss_feat = cyp2d6_smiles_merge.merge(cyp2d6_mol, how="inner", on="index")

# Save to data/cyp_datasets
cyp2d6_swiss_feat.to_pickle('.././data/cyp_datasets/cyp2d6_swiss_feat.pkl')
cyp2d6_swiss_feat.head()

<class 'pandas.core.frame.DataFrame'>


,index,Inhibition Observed,Panel Name,h_bond_donors,h_bond_acceptors,molar_refractivity,molecular_weight,n_atoms,n_carbons,n_heteroatoms,n_rings,n_rot_bonds,logp,tpsa,n_heavy_atoms,n_aromatic_atom,single_bond,double_bond,triple_bond,aromatic_bond
0,6,1,2,0.0,4.0,66.0385,246.089209,18.0,14.0,4.0,2.0,6.0,3.06980,48.67,18.0,11.0,14.0,2.0,0.0,22.0
1,11,1,2,0.0,3.0,79.6170,316.021125,19.0,15.0,4.0,3.0,2.0,4.08082,26.53,19.0,15.0,10.0,0.0,0.0,32.0
2,16,1,2,1.0,4.0,80.6693,337.103811,24.0,16.0,8.0,3.0,2.0,2.95000,57.83,24.0,11.0,26.0,4.0,0.0,22.0
3,21,1,2,1.0,4.0,90.3577,315.086390,21.0,16.0,5.0,3.0,4.0,4.84089,33.61,21.0,16.0,12.0,2.0,0.0,32.0
4,26,1,2,0.0,6.0,65.7749,268.117155,19.0,11.0,8.0,2.0,2.0,0.57770,90.50,19.0,5.0,26.0,4.0,0.0,10.0


## CYp1a2

In [8]:
# CYP2c19 is mapped to the 3rd value
cyp1a2_smiles_merge = smiles_merged[smiles_merged['Panel Name'] == 3]
cyp1a2_smiles = cyp1a2_smiles_merge[['SMILES']]

# Convert MolFromSmiles to GetMorganFingerprintAsBitVect to ToBitString to get bitstrings for SMILES data
cyp1a2_mol = cyp1a2_smiles.apply(lambda row: Chem.MolFromSmiles(row['SMILES']), axis=1)
cyp1a2_mol = cyp1a2_mol.to_list()

# Set up mol dataset with index
cyp1a2_mol = swiss_feat(cyp1a2_mol)
cyp1a2_mol = cyp1a2_mol.reset_index()

print(type(cyp1a2_mol))

#combine datasets for modeling
cyp1a2_smiles_merge = cyp1a2_smiles_merge[['index', 'Inhibition Observed', 'Panel Name']]
cyp1a2_swiss_feat = cyp1a2_smiles_merge.merge(cyp1a2_mol, how="inner", on="index")

# Save to data/cyp_datasets
cyp1a2_swiss_feat.to_pickle('.././data/cyp_datasets/cyp1a2_swiss_feat.pkl')
cyp1a2_swiss_feat.head()

<class 'pandas.core.frame.DataFrame'>


,index,Inhibition Observed,Panel Name,h_bond_donors,h_bond_acceptors,molar_refractivity,molecular_weight,n_atoms,n_carbons,n_heteroatoms,n_rings,n_rot_bonds,logp,tpsa,n_heavy_atoms,n_aromatic_atom,single_bond,double_bond,triple_bond,aromatic_bond
0,8,1,3,1.0,6.0,74.6722,291.067762,20.0,13.0,7.0,3.0,3.0,2.12400,73.34,20.0,11.0,20.0,2.0,0.0,22.0
1,13,1,3,1.0,4.0,82.4302,292.070405,19.0,14.0,5.0,2.0,5.0,3.83442,41.46,19.0,10.0,16.0,4.0,0.0,20.0
2,18,1,3,1.0,4.0,82.1107,291.086390,19.0,14.0,5.0,3.0,3.0,3.97999,33.61,19.0,10.0,18.0,4.0,0.0,20.0
3,23,1,3,1.0,5.0,64.6447,249.057198,17.0,11.0,6.0,3.0,2.0,2.35629,52.07,17.0,11.0,14.0,2.0,0.0,22.0
4,28,1,3,2.0,2.0,81.2985,327.042899,21.0,15.0,6.0,2.0,4.0,3.27660,66.40,21.0,6.0,26.0,6.0,0.0,12.0


## CYP3a4

In [9]:
# CYP2c19 is mapped to the 4th value
cyp3a4_smiles_merge = smiles_merged[smiles_merged['Panel Name'] == 4]
cyp3a4_smiles = cyp3a4_smiles_merge[['SMILES']]

# Convert MolFromSmiles to GetMorganFingerprintAsBitVect to ToBitString to get bitstrings for SMILES data
cyp3a4_mol = cyp3a4_smiles.apply(lambda row: Chem.MolFromSmiles(row['SMILES']), axis=1)
cyp3a4_mol = cyp3a4_mol.to_list()

# Set up mol dataset with index
cyp3a4_mol = swiss_feat(cyp3a4_mol)
cyp3a4_mol = cyp3a4_mol.reset_index()

print(type(cyp3a4_mol))

#combine datasets for modeling
cyp3a4_smiles_merge = cyp3a4_smiles_merge[['index', 'Inhibition Observed', 'Panel Name']]
cyp3a4_swiss_feat = cyp3a4_smiles_merge.merge(cyp3a4_mol, how="inner", on="index")

# Save to data/cyp_datasets
cyp3a4_swiss_feat.to_pickle('.././data/cyp_datasets/cyp3a4_swiss_feat.pkl')
cyp3a4_swiss_feat.head()

<class 'pandas.core.frame.DataFrame'>


,index,Inhibition Observed,Panel Name,h_bond_donors,h_bond_acceptors,molar_refractivity,molecular_weight,n_atoms,n_carbons,n_heteroatoms,n_rings,n_rot_bonds,logp,tpsa,n_heavy_atoms,n_aromatic_atom,single_bond,double_bond,triple_bond,aromatic_bond
0,7,1,4,0.0,5.0,74.6390,275.072848,19.0,13.0,6.0,3.0,1.0,1.3841,54.79,19.0,6.0,24.0,6.0,0.0,12.0
1,12,1,4,1.0,4.0,71.8922,260.007805,17.0,12.0,5.0,3.0,2.0,3.6101,41.99,17.0,14.0,6.0,2.0,0.0,30.0
2,17,1,4,1.0,3.0,67.5397,269.982631,14.0,10.0,4.0,2.0,2.0,2.9648,24.39,14.0,6.0,16.0,2.0,0.0,12.0
3,22,1,4,3.0,4.0,54.8078,224.079707,16.0,10.0,6.0,1.0,4.0,-0.3546,99.52,16.0,6.0,16.0,4.0,0.0,12.0
4,27,1,4,0.0,6.0,65.7749,268.117155,19.0,11.0,8.0,2.0,2.0,0.5777,90.50,19.0,5.0,26.0,4.0,0.0,10.0


In [10]:
df_list = [cyp2c19_swiss_feat, cyp2c9_swiss_feat, cyp2d6_swiss_feat, cyp1a2_swiss_feat, cyp3a4_swiss_feat]
cyp_sw_feat_merged = pd.concat(df_list)
cyp_sw_feat_merged.to_pickle('../data/merged_dfs/cyp_sw_feat_merged.pkl')